<a href="https://colab.research.google.com/github/physicaone/CANonEQal/blob/master/%5B210516%5Dsampling_with_PT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    base='Google Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.utils.data
import torch.nn as nn
from itertools import combinations
from itertools import permutations
from datetime import datetime
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
# np.seterr(divide='ignore', invalid='ignore')
import itertools
import re
from time import sleep

#Define RBM class

In [42]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k, use_cuda):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        if use_cuda==True:
            self.v = nn.Parameter(torch.ones(1, n_vis).cuda())
            self.h = nn.Parameter(torch.zeros(1, n_hid).cuda())
            self.W = nn.Parameter(torch.randn(n_hid, n_vis).cuda())
            self.k = k
        else:
            self.v = nn.Parameter(torch.ones(1, n_vis))
            self.h = nn.Parameter(torch.zeros(1, n_hid))
            self.W = nn.Parameter(torch.randn(n_hid, n_vis))
            self.k = k     

    def visible_to_hidden(self, v, beta):
        return torch.sigmoid(F.linear(v, self.W, self.h)*beta)

    def hidden_to_visible(self, h, beta):
        return torch.sigmoid(F.linear(h, self.W.t(), self.v)*beta)

    def free_energy(self, v):
        v_term = torch.matmul(v, self.v.t())
        w_x_h = F.linear(v, self.W, self.h)
        h_term = torch.sum(F.softplus(w_x_h), dim=1)
        return torch.mean(-h_term - v_term)
    
    def energy2(self, v, h):
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())

    def forward(self, v):
        h = self.visible_to_hidden(v)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibb = self.hidden_to_visible(h)
            v_gibb = v_gibb.bernoulli()
            h = self.visible_to_hidden(v_gibb)
            h = h.bernoulli()
        return v, v_gibb

# Check CUDA availability

In [43]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available(); CUDA

False

# Define fundamental functions

In [49]:
# 아래 두 함수는 PT와는 아무 상관 없습니다. 10진법을 2진법으로 바꾸고 리스트 형태로 변형하는 함수입니다. 예)15->[1,1,1,1]
# 그리고 그 다음은 역함수입니다.
def decimal_to_binary(integer):
    string=bin(integer)[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def binary_to_decimal(list0):
    value=0
    for i in range(len(list0)):
        value+=list0[-i-1]*2**(i)
    return value

# 이 함수는 PT에 사용되는 transition 확률을 계산합니다.
def swap_prob(i,j, model, list00, list11):
    v1=torch.tensor(list00[i]).view(1,n_vis)
    v2=torch.tensor(list00[j]).view(1,n_vis)
    h1=torch.tensor(list11[i]).view(1,n_hid)
    h2=torch.tensor(list11[j]).view(1,n_hid)
    beta1=beta_list[i]
    beta2=beta_list[j]
    return torch.exp((beta2-beta1)*(model.energy2(v2, h2)-model.energy2(v1, h1)))

# 이 함수는 tansition 확률에 의거, swap을 수행합니다.
def swap(list0, list1, model):
    k=np.random.randint(0, len(list0)-1)
    combination=[k, k+1]
    if swap_prob(combination[0], combination[1], model, list0, list1)>=np.random.rand():
        a=list0[combination[0]]
        b=list0[combination[1]] 
        list0[combination[0]]=b
        list0[combination[1]]=a
        
        c=list1[combination[0]]
        d=list1[combination[1]] 
        list1[combination[0]]=d
        list1[combination[1]]=c
    else: None
#     return list0, list1
#     combinations=list(itertools.combinations(list(range(len(list0))), 2))
#     for k in range(len(combinations)):
#         if swap_prob(combinations[k][1], combinations[k][0])>=np.random.rand():
#             list0[combinations[k][0]]=list0[combinations[k][1]]; list0[combinations[k][1]]=list0[combinations[k][0]]
#         else: None
#     return list0

def P_h(list0):
    config_count={}
    for i in range(len(list0)):
        config_count[str(int(binary_to_decimal(list0[i][0])))]=0
    for i in range(len(list0)):
        config_count[str(int(binary_to_decimal(list0[i][0])))]+=1
    return config_count

def prod(L):
    p=1
    for i in L:
        p= i * p
    return p

# def Estimate_Z(model0, states):
#     Z=0
#     for i in range(len(states[0])):
#         Z+=torch.exp(-model0.energy2(states[0][i], states[1][i])).detach()
#     return float(Z.detach().numpy())

def get_hist(list00, color='red'):
    bins=range(int(min(list00)-30), int(max(list00)+30), 1)
    y1,x1,_ = plt.hist(list00, bins = bins, histtype='step', color=color)
    x1 = 0.5*(x1[1:]+x1[:-1])
    return x1, y1

def flatten_list(list0):
    flattened = [val for sublist in list0 for val in sublist]
    return flattened


# Define stat. physical functions

In [46]:
def Entropy(fullconfigs):
    config_count={} # 각 hidden layer state 갯수 파악 (k)
    for i in range(len(fullconfigs)):
        config_count[str(fullconfigs[i])]=0
    for i in range(len(fullconfigs)):
        config_count[str(fullconfigs[i])]+=1

    listk=[]
    for i in range(len(list(config_count.values()))):
        listk.append(int(list(config_count.values())[i]))
    listmk=[]
    kcount={} # 갯수의 갯수 파악 (m_k)
    for i in range(len(listk)):
        kcount[listk[i]]=0
    for i in range(len(listk)):
        kcount[listk[i]]+=1
    for i in range(len(kcount)):
        listmk.append(kcount[sorted(list(kcount))[i]])
    x,y= sorted(list(kcount)), listmk

    list100=[]
    list100kmk=[]
    for i in range(len(x)):
        list100kmk.append(x[i]*y[i])
    for i in range(len(x)):
        for j in range(list100kmk[i]):
            list100.append(x[i])
    N=len(list100)
    H_s=0
    for i in range(len(x)):
        H_s-=(x[i]*y[i]/N)*np.log(x[i]/N)
    return H_s

def FE_mean_std(model0_dict, p):
    FE_list=[]
    for i in range(len(p)):
        # n_vis=len(model0_dict['v'][0]); n_hid=len(model0_dict['h'][0])
        integer=list(p.keys())[i]
        h=decimal_to_binary(int(integer)).to(device)
        a=model0_dict['v'].view(1,n_vis).detach()
        b=model0_dict['h'].view(1,n_hid).detach()
        W=model0_dict['W'].view(n_hid,n_vis).detach()
        
        FE=prod(torch.sigmoid(-F.linear(h, W.t(), a)).detach()[0])*100000
        FE=FE*p[integer]
        FE=FE/sum(list(p.values()))
        FE=torch.log(FE.detach())-torch.log(torch.tensor([[100000]]))-torch.dot(b[0],h[0].t())
        FE_list.append(FE.detach())
    return [float(torch.mean(torch.tensor(FE_list)).detach().numpy()), float(torch.std(torch.tensor(FE_list)).detach().numpy())]

def dE_mean_std(model0_dict, model1_dict, v_list, h_list):
    dE=[]
    for i in range(len(v_list)):
        E0=model0_dict.energy2(v_list[i].float().detach(), h_list[i].float().detach())[0].detach()
        E1=model1_dict.energy2(v_list[i].float().detach(), h_list[i].float().detach())[0].detach()
        dE.append(E1-E0)
    return [float(torch.mean(torch.tensor(dE)).numpy()), float(torch.std(torch.tensor(dE)).numpy())]

def Energy(model0_dict, v_list, h_list):
    a=model0_dict['v'].detach()
    b=model0_dict['h'].detach()
    W=model0_dict['W'].detach()
    values=[]
    for i in range(len(v_list)):
        e=-np.matmul(v_list[i], a.t())-np.matmul(np.matmul(v_list[i], W.t()), h_list[i].t())-np.matmul(h_list[i], b.t())
        values.append(e.detach())
    return float(np.mean(values))

def FE_V(model0, v0):
    v0=torch.tensor(v0)
    a=model0['v'].detach()
    b=model0['h'].detach()
    W=model0['W'].detach()
    values=[]
    for i in range(len(v0)):
        v_term = torch.matmul(v0[i].detach(), a.t()).detach()
        w_x_h = F.linear(v0[i].detach(), W, b).detach()
        h_term = torch.sum(F.softplus(w_x_h), dim=1).detach()
        values.append(torch.mean(-h_term - v_term).detach().numpy())
    return float(np.mean(values))
    
def sampling_with_PT(fullmodel, eq_step, n_step):
    # states for full model
    hidden_states_now0=[decimal_to_binary(2**n_hid-1)]*len(beta_list)
    visible_states_now0=[1.]*len(beta_list)

    coldest_hidden0=[]
    coldest_visible0=[]
    
    # equilibrium steps for full model
    for step in range(eq_step):
        for i in range(len(beta_list)):
            visible_states_now0[i]=fullmodel.hidden_to_visible(hidden_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
            hidden_states_now0[i]=fullmodel.visible_to_hidden(visible_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
    

    # Tasks with PT
    for step in range(n_step):
        for i in range(len(beta_list)):
            # Gibbs sampling of fu
            visible_states_now0[i]=fullmodel.hidden_to_visible(hidden_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
            hidden_states_now0[i]=fullmodel.visible_to_hidden(visible_states_now0[i].to(device), beta_list[i]).bernoulli().detach()
        swap(visible_states_now0, hidden_states_now0, fullmodel)
        coldest_hidden0.append(hidden_states_now0[0].int())
        coldest_visible0.append(visible_states_now0[0].int())

    return coldest_visible0, coldest_hidden0

# mn model의 샘플 만들기


In [ ]:
beta_list=torch.tensor(np.linspace(1,0,10).astype(int)).to(device)
n_vis=9
T='3.2'
for n_hid in [1,2,4,8]:
    models=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn.pkl'.format(base=base, n_hid=n_hid, T=T))
    for m in tnrange(10):
        for n in range(10):
            list_v=[]; list_h=[]
            model1_dict=models[str(m)][str(n)]
            model1=RBM(n_vis=n_vis, n_hid=n_hid, k=1, use_cuda=CUDA)
            model1.load_state_dict(model1_dict)
            a0, b0=sampling_with_PT(model1, 100, 100000)
            list_v.append(a0); list_h.append(b0)
            with open('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_m={m}_n={n}.pkl'.format(base=base, n_hid=n_hid, T=T, m=m, n=n), 'wb') as f:
                pkl.dump([list_v, list_h], f)

# CM_m model의 샘플 만들기


In [47]:
beta_list=torch.tensor(np.linspace(1,0,10).astype(int)).to(device)
n_vis=9
T='3.2'
for n_hid in [1,2,4,8]:
    models=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_m.pkl'.format(base=base, n_hid=n_hid, T=T))
    for m in tnrange(10):
        list_v=[]; list_h=[]
        model1_dict=models[str(m)]
        model1=RBM(n_vis=n_vis, n_hid=n_hid, k=1, use_cuda=CUDA)
        model1.load_state_dict(model1_dict)
        a0, b0=sampling_with_PT(model1, 100, 100000)
        list_v.append(a0); list_h.append(b0)
        with open('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM_m={m}.pkl'.format(base=base, n_hid=n_hid, T=T, m=m), 'wb') as f:
            pkl.dump([list_v, list_h], f)

# CM model의 샘플 만들기

In [48]:
beta_list=torch.tensor(np.linspace(1,0,10).astype(int)).to(device)
n_vis=9
T='3.2'
for n_hid in [1,2,4,8]:
    list_v=[]; list_h=[]
    model1_dict=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T))
    
    model1=RBM(n_vis=n_vis, n_hid=n_hid, k=1, use_cuda=CUDA)
    model1.load_state_dict(model1_dict)
    a0, b0=sampling_with_PT(model1, 100, 100000)
    list_v.append(a0); list_h.append(b0)
    with open('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
        pkl.dump([list_v, list_h], f)

# Entropy of m

In [ ]:
T='3.2'
fullconfigs=pd.read_pickle('{base}/loss_IG/3*3/3*3_full_T=3.2.pkl'.format(base=base))
S_m={}
for m in range(10):
    S_m[str(m)]=Entropy(fullconfigs[m])
S_CM=Entropy(flatten_list(fullconfigs))
with open('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_T={T}_S.pkl'.format(base=base, T=T), 'wb') as f:
    pkl.dump({'CM_m':S_m, 'CM':S_CM}, f)

# CM, CM_m, mn의 Free energy 계산

In [50]:
T='3.2'
n_vis=9
for n_hid in [1,2,4,8]:
    dict0={}; dict1={}; dict2={}
    model_dicts_mn=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn.pkl'.format(base=base, n_hid=n_hid, T=T))
    model_dicts_CM_m=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_m.pkl'.format(base=base, n_hid=n_hid, T=T))
    model_dicts_CM=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T))

    for m in tnrange(10):
        dict0[str(m)]={}
        for n in range(10):
            vh0=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_m={m}_n={n}.pkl'.format(base=base, n_hid=n_hid,T=T,m=m,n=n))
            model_dict0=model_dicts_mn[str(m)][str(n)]
            dict0[str(m)][str(n)]=FE_mean_std(model_dict0, P_h(vh0[1][0]))[0]
        vh1=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM_m={m}.pkl'.format(base=base, n_hid=n_hid,T=T,m=m))
        model_dict1=model_dicts_CM_m[str(m)]
        dict1[str(m)]=FE_mean_std(model_dict1, P_h(vh1[1][0]))[0]
    vh2=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid,T=T))
    model_dict2=model_dicts_CM
    dict2=FE_mean_std(model_dict2,
    P_h(vh2[1][0]))[0]
    with open('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
        pkl.dump({'mn':dict0, 'CM_m':dict1, 'CM':dict2}, f)

# CM, CM_m, mn의 marginalized Free energy 계산

In [54]:
fullconfigs=pd.read_pickle('{base}/loss_IG/3*3/3*3_full_T=3.2.pkl'.format(base=base))
T='3.2'
n_vis=9
for n_hid in [1,2,4,8]:
    dict0={}; dict1={}; dict2={}
    model_dicts_mn=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn.pkl'.format(base=base, n_hid=n_hid, T=T))
    model_dicts_CM_m=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_m.pkl'.format(base=base, n_hid=n_hid, T=T))
    model_dicts_CM=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T))

    for m in tnrange(10):
        dict0[str(m)]={}
        for n in range(10):
            vh0=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_m={m}_n={n}.pkl'.format(base=base, n_hid=n_hid,T=T,m=m,n=n))
            model_dict0=model_dicts_mn[str(m)][str(n)]
            dict0[str(m)][str(n)]=FE_V(model_dict0, fullconfigs[m])
        vh1=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM_m={m}.pkl'.format(base=base, n_hid=n_hid,T=T,m=m))
        model_dict1=model_dicts_CM_m[str(m)]
        dict1[str(m)]=FE_V(model_dict1, fullconfigs[m])
    vh2=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid,T=T))
    model_dict2=model_dicts_CM
    dict2=FE_V(model_dict2, flatten_list(fullconfigs))
    with open('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE_V.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
        pkl.dump({'mn':dict0, 'CM_m':dict1, 'CM':dict2}, f)

# CM, CM_m, mn의 <E'-E> 계산

In [ ]:
T='3.2'
for n_hid in [1,2,4,8]:
    dE_CM={}; dE_CM_m={}
    model_dict_CM=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T))
    model_dict_CM_m=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_CM_m.pkl'.format(base=base, n_hid=n_hid, T=T))
    model_dict_mn=pd.read_pickle('{base}/loss_IG/3*3/state_dict/n_hid={n_hid}_T={T}_mn.pkl'.format(base=base, n_hid=n_hid, T=T))
    vh=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM.pkl'.format(base=base, n_hid=n_hid, T=T))
    for m in tnrange(10):
        dE_CM[str(m)]={}; dE_CM_m[str(m)]={}
        vh_m=pd.read_pickle('{base}/loss_IG/3*3/samples/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_CM_m={m}.pkl'.format(base=base, n_hid=n_hid, T=T, m=m))
        for n in range(10):
            dE_CM[str(m)][str(n)]=Energy(model_dict_mn[str(m)][str(n)], vh[0][0], vh[1][0])-Energy(model_dict_CM, vh[0][0], vh[1][0])
            dE_CM_m[str(m)][str(n)]=Energy(model_dict_mn[str(m)][str(n)], vh_m[0][0], vh_m[1][0])-Energy(model_dict_CM_m[str(m)], vh_m[0][0], vh_m[1][0])
    with open('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_dE.pkl'.format(base=base, n_hid=n_hid, T=T), 'wb') as f:
        pkl.dump({'CM':dE_CM, 'CM_m':dE_CM_m}, f)

# Calculate bias moments

In [140]:
T='3.2'
S=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_T={T}_S.pkl'.format(base=base, T=T))
for n_hid in [1,2,4,8]:
    bias_2nd_moment1=0; bias_2nd_moment2=0
    FE=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE.pkl'.format(base=base, n_hid=n_hid, T=T))
    FE_V=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE_V.pkl'.format(base=base, n_hid=n_hid, T=T))
    FE_CM=FE['CM']; FE_CM_m=FE['CM_m']; FE_mn=FE['mn']
    FE_V_CM=FE['CM']; FE_V_CM_m=FE['CM_m']; FE_V_mn=FE['mn']
    bias_CM=-S['CM']-FE_CM+FE_V_CM
    for m in range(10):
        bias_CM_m=-S['CM_m'][str(m)]-FE_CM_m[str(m)]+FE_V_CM_m[str(m)]
        for n in range(10):
            bias_tmp_mn=-S['CM']-FE_mn[str(m)][str(n)]+FE_V_mn[str(m)][str(n)]
            bias_2nd_moment1+=(bias_CM-bias_tmp_mn)**2 
            bias_2nd_moment2+=(bias_CM_m-bias_tmp_mn)**2
    print(n_hid, bias_2nd_moment1/100, bias_2nd_moment2/100)

1 6.310887241768095e-32 7.767157794213993e-05
2 1.7749370367472765e-32 7.767157794214104e-05
4 0.0 7.767157794214155e-05
8 0.0 7.767157794214155e-05


# Calculate variances

In [141]:
T='3.2'
for n_hid in [1,2,4,8]:
    variance1=0; variance2=0
    dE=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_dE.pkl'.format(base=base, n_hid=n_hid, T=T))
    FE=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE.pkl'.format(base=base, n_hid=n_hid, T=T))
    FE_CM=FE['CM']; FE_CM_m=FE['CM_m']; FE_mn=FE['mn']
    for m in range(10):
        for n in range(10):
            variance1=+dE['CM'][str(m)][str(n)]-FE_mn[str(m)][str(n)]-FE_CM
            variance2=+dE['CM_m'][str(m)][str(n)]-FE_mn[str(m)][str(n)]-FE_CM_m[str(m)]
    print(n_hid, variance1/100, variance2/100)

1 0.1384396411623478 0.1411980746648699
2 0.1498523440063268 0.14999341785370185
4 0.1723194813623339 0.17332492362703086
8 0.2012110901477579 0.1962218525152616


In [136]:
T='3.2'


n_hid=8
dict0={}
for m in range(10):
    dict0[str(m)]=[]
    for n in range(10):
        value=[]
        for n_hid in [1,2,4,8]:
            FE_mn=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE.pkl'.format(base=base, n_hid=n_hid, T=T))['mn']
            FE_V_mn=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE_V.pkl'.format(base=base, n_hid=n_hid, T=T))['mn']
            S=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_T={T}_S.pkl'.format(base=base, T=T))
            value.append(np.round(-S['CM_m'][str(m)]-FE_mn[str(m)][str(n)]+FE_V_mn[str(m)][str(n)],3))
        dict0[str(m)].append(value)
df = pd.DataFrame(data=dict0)


In [137]:
df.T

,0,1,2,3,4,5,6,7,8,9
0,"[3.472, 3.414, 2.978, 2.582]","[3.362, 3.432, 2.866, 2.92]","[3.264, 3.393, 3.404, 2.914]","[3.474, 3.108, 2.971, 3.051]","[3.483, 3.187, 2.749, 2.9]","[3.265, 3.119, 3.261, 2.929]","[3.285, 3.397, 3.135, 2.715]","[3.248, 3.243, 3.254, 2.946]","[3.284, 3.237, 3.039, 2.836]","[3.461, 3.279, 3.151, 3.063]"
1,"[3.48, 3.024, 3.336, 2.936]","[3.434, 3.243, 3.099, 2.56]","[3.421, 3.341, 3.137, 2.959]","[3.417, 3.261, 3.088, 3.011]","[3.497, 3.414, 3.213, 3.049]","[3.452, 3.216, 3.247, 2.638]","[3.507, 3.361, 2.966, 2.885]","[3.33, 3.428, 2.955, 2.814]","[3.445, 3.404, 3.088, 2.799]","[3.397, 3.458, 2.885, 2.878]"
2,"[3.483, 3.33, 3.028, 2.819]","[3.288, 3.207, 3.097, 2.718]","[3.51, 3.175, 3.123, 2.707]","[3.328, 3.155, 3.156, 2.953]","[3.374, 3.207, 3.182, 2.117]","[3.49, 3.405, 2.87, 2.77]","[3.47, 2.988, 3.01, 3.038]","[3.447, 3.287, 2.925, 2.458]","[3.302, 3.327, 3.125, 2.823]","[3.466, 3.295, 3.066, 2.539]"
3,"[3.398, 3.439, 2.995, 2.67]","[3.267, 3.035, 3.022, 2.701]","[3.264, 3.209, 2.962, 2.664]","[3.464, 3.176, 3.205, 2.62]","[3.347, 3.299, 3.014, 2.732]","[3.461, 3.2, 2.658, 2.743]","[3.442, 3.344, 2.906, 2.896]","[3.275, 3.207, 3.025, 2.884]","[3.408, 3.307, 3.231, 2.765]","[3.341, 3.386, 2.858, 2.713]"
4,"[3.283, 3.181, 3.244, 3.011]","[3.456, 3.339, 2.965, 2.765]","[3.373, 3.232, 3.119, 2.539]","[3.438, 3.232, 2.903, 2.687]","[3.429, 3.32, 3.162, 2.872]","[3.362, 3.144, 3.194, 2.795]","[3.449, 3.31, 3.206, 2.588]","[3.451, 3.095, 2.832, 2.521]","[3.405, 3.27, 3.124, 2.655]","[3.466, 3.252, 3.056, 2.796]"
5,"[3.253, 3.476, 2.92, 2.974]","[3.335, 3.31, 3.099, 2.816]","[3.385, 3.179, 2.911, 2.855]","[3.243, 3.121, 3.12, 2.873]","[3.392, 3.208, 2.926, 2.889]","[3.422, 3.167, 2.983, 2.8]","[3.417, 3.312, 3.076, 2.907]","[3.243, 3.225, 2.963, 2.877]","[3.448, 3.245, 2.995, 2.951]","[3.265, 3.182, 3.064, 3.013]"
6,"[3.282, 3.348, 3.114, 2.568]","[3.255, 3.136, 2.894, 3.015]","[3.452, 3.441, 3.244, 2.791]","[3.319, 3.436, 3.157, 3.01]","[3.331, 3.221, 3.025, 2.751]","[3.503, 3.263, 3.105, 2.685]","[3.339, 3.312, 3.051, 3.027]","[3.506, 3.36, 3.035, 2.788]","[3.421, 3.353, 3.146, 2.563]","[3.4, 3.235, 3.256, 2.794]"
7,"[3.299, 3.234, 3.184, 2.703]","[3.49, 3.411, 2.88, 3.132]","[3.371, 3.419, 3.322, 2.71]","[3.295, 3.431, 2.862, 2.751]","[3.356, 3.307, 3.248, 2.702]","[3.477, 3.357, 3.085, 2.735]","[3.459, 3.171, 3.055, 2.969]","[3.328, 3.186, 2.911, 2.876]","[3.442, 3.211, 2.996, 2.941]","[3.275, 3.329, 3.022, 2.766]"
8,"[3.483, 3.219, 3.089, 2.784]","[3.455, 3.365, 3.178, 2.752]","[3.478, 3.44, 3.19, 3.084]","[3.348, 3.394, 3.009, 3.033]","[3.375, 3.275, 3.121, 2.674]","[3.297, 3.419, 3.149, 3.117]","[3.353, 3.386, 3.035, 2.784]","[3.277, 3.162, 3.021, 2.699]","[3.481, 3.19, 3.236, 2.706]","[3.453, 3.328, 3.149, 2.896]"
9,"[3.352, 3.429, 2.869, 2.749]","[3.399, 3.366, 2.916, 2.737]","[3.24, 3.232, 3.369, 2.948]","[3.379, 3.214, 3.071, 2.92]","[3.45, 3.103, 3.145, 2.89]","[3.448, 3.385, 2.97, 2.904]","[3.211, 3.117, 3.013, 3.011]","[3.19, 3.232, 3.161, 2.497]","[3.172, 3.178, 2.978, 3.137]","[3.2, 3.388, 3.107, 3.17]"


In [128]:
T='3.2'

n_hid=1
FE_mn=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE.pkl'.format(base=base, n_hid=n_hid, T=T))
dE=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_dE.pkl'.format(base=base, n_hid=n_hid, T=T))
dict_CM_m={}; dict_CM={}
for m in range(10):
    dict_CM_m[str(m)]=[]
    dict_CM[str(m)]=[]
    for n in range(10):
        value1=[]; value2=[]
        for n_hid in [1,2,4,8]:
            FE_mn=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_FE.pkl'.format(base=base, n_hid=n_hid, T=T))
            dE=pd.read_pickle('{base}/loss_IG/3*3/data/3*3_PT10_0.1M_n_hid={n_hid}_T={T}_dE.pkl'.format(base=base, n_hid=n_hid, T=T))
            value1.append(np.round(dE['CM_m'][str(m)][str(n)]-FE['mn'][str(m)][str(n)]+FE['CM_m'][str(m)],3))
            value2.append(np.round(dE['CM'][str(m)][str(n)]-FE['mn'][str(m)][str(n)]+FE['CM'],3))
        dict_CM_m[str(m)].append(value1)
        dict_CM[str(m)].append(value2)
df_CM = pd.DataFrame(data=dict_CM); df_CM_m = pd.DataFrame(data=dict_CM_m) 

In [129]:
df_CM.T

,0,1,2,3,4,5,6,7,8,9
0,"[0.065, 0.1, -0.925, -0.476]","[0.029, 0.867, -0.661, -0.088]","[0.067, 1.341, 2.143, 0.968]","[0.051, -0.926, -0.781, 0.759]","[0.108, -1.244, -0.735, -1.14]","[0.064, 0.258, 1.978, 2.317]","[0.056, 1.041, 0.039, 0.482]","[0.049, 0.866, 1.866, 0.86]","[0.057, -0.342, 0.018, 0.332]","[0.061, -0.69, -0.324, 0.072]"
1,"[0.079, -1.202, 0.713, -0.275]","[0.087, -0.763, -0.384, -0.76]","[0.047, 0.382, 0.02, -0.1]","[0.154, -1.112, -1.509, -0.288]","[0.087, -0.472, -0.328, -0.439]","[0.033, -0.35, 1.446, -0.458]","[0.077, 0.203, -0.776, 0.121]","[0.073, 1.102, -0.126, -0.527]","[0.069, 1.297, 0.582, 0.495]","[0.209, -0.597, -1.733, -1.975]"
2,"[0.092, -0.585, -0.382, -0.168]","[0.063, 0.074, 0.536, -0.002]","[0.067, -0.704, -0.391, -0.253]","[0.067, -0.361, 0.965, 1.818]","[0.052, -0.06, 0.532, -0.139]","[0.135, -0.471, -1.349, -2.293]","[0.043, -1.348, -0.543, 0.169]","[0.034, 0.308, -0.329, -0.384]","[0.085, 1.848, 2.058, 1.127]","[0.133, -1.265, -2.228, -0.994]"
3,"[0.024, 1.115, -0.932, -0.906]","[0.058, 0.042, 0.366, 1.082]","[0.056, 0.702, 0.86, 0.31]","[0.035, -0.307, 0.657, -0.257]","[0.058, 0.703, 0.006, 0.54]","[0.036, -1.004, -1.115, 0.393]","[0.029, -0.218, -0.312, -0.302]","[0.075, 0.532, 0.244, 0.912]","[0.034, -0.021, 1.272, 0.143]","[0.033, 1.922, -0.412, -0.243]"
4,"[0.07, 0.818, 1.359, 0.661]","[0.043, 0.319, -0.201, -0.009]","[0.048, -0.052, 1.263, -0.406]","[0.139, -1.104, -1.505, -1.525]","[0.025, 0.409, -0.052, -0.028]","[0.047, -0.448, 1.719, 0.851]","[0.166, -1.713, -1.425, -1.884]","[0.075, -0.942, -1.259, -0.489]","[0.017, 0.209, 1.476, 0.306]","[0.163, -1.468, -1.577, -1.612]"
5,"[0.068, 1.858, 0.269, 1.878]","[0.03, 0.087, -0.32, -0.025]","[0.04, -0.348, 0.151, -0.569]","[0.08, 0.694, 1.028, 1.611]","[0.033, -0.574, -0.967, -0.172]","[0.041, -0.408, 0.25, 0.427]","[0.042, -0.363, -0.663, 0.31]","[0.069, 0.698, 0.948, 0.977]","[0.123, -1.7, -1.143, -1.644]","[0.08, 0.059, 0.483, -0.514]"
6,"[0.073, 1.095, 1.184, -0.109]","[0.075, 0.542, 0.112, 0.509]","[0.136, 1.01, 0.086, -0.933]","[0.084, 0.933, 0.823, 0.375]","[0.102, -0.347, -0.181, 0.267]","[0.092, -0.689, 0.342, 0.191]","[0.061, 0.744, 0.192, 1.426]","[0.076, 0.568, -0.204, 0.423]","[0.056, 0.032, 0.484, 0.509]","[0.021, -0.313, 1.633, 0.184]"
7,"[0.095, 0.401, 1.126, 0.953]","[0.118, -1.125, -1.585, -0.069]","[0.041, 0.869, 2.683, 0.623]","[0.06, 1.618, -0.409, -0.412]","[0.039, 0.285, 0.653, 0.39]","[0.052, 0.574, -0.597, -0.052]","[0.125, -1.386, -0.404, -2.027]","[0.052, 0.053, 0.583, 0.98]","[0.043, -0.415, -0.034, 0.2]","[0.085, 1.386, 0.537, 0.796]"
8,"[0.05, -0.313, -0.63, 0.35]","[0.137, -1.569, -1.417, -1.894]","[0.09, -0.48, -1.079, -0.089]","[0.053, 0.331, -0.01, 0.499]","[0.067, 0.654, -0.052, 0.5]","[0.067, 1.949, 0.991, 1.4]","[0.05, 1.003, 0.065, -0.356]","[0.107, 0.496, 0.358, 0.845]","[0.038, -0.383, 0.764, -0.734]","[0.091, 0.791, -0.133, -1.329]"
9,"[0.05, 1.134, 0.551, -0.301]","[0.062, 0.289, -1.235, -1.271]","[0.067, 0.127, 2.199, 0.224]","[0.062, 0.468, -0.404, 0.429]","[0.06, -0.707, -0.277, -0.653]","[0.095, -0.513, -0.927, -0.102]","[0.051, 0.034, 0.6, 0.429]","[0.025, 0.141, 0.275, 1.005]","[0.008, 0.003, -0.448, 2.521]","[0.039, 0.902, 0.813, 2.616]"


In [130]:
df_CM_m.T

,0,1,2,3,4,5,6,7,8,9
0,"[0.076, 0.051, -1.128, -0.851]","[0.042, 0.81, -0.898, -0.502]","[0.057, 1.299, 1.988, 0.598]","[0.069, -0.983, -0.991, 0.462]","[0.129, -1.318, -0.961, -1.542]","[0.054, 0.205, 1.852, 2.015]","[0.062, 0.975, -0.143, 0.201]","[0.044, 0.778, 1.63, 0.527]","[0.066, -0.403, -0.192, 0.061]","[0.066, -0.749, -0.494, -0.303]"
1,"[0.052, -0.933, 1.009, 0.199]","[0.054, -0.5, -0.123, -0.337]","[0.045, 0.572, 0.301, 0.328]","[0.069, -0.936, -1.21, 0.381]","[0.058, -0.242, -0.034, -0.028]","[0.029, -0.124, 1.671, 0.112]","[0.058, 0.417, -0.5, 0.592]","[0.085, 1.3, 0.156, -0.017]","[0.082, 1.506, 0.82, 0.976]","[0.124, -0.403, -1.433, -1.549]"
2,"[0.057, -0.255, -0.215, 0.038]","[0.061, 0.43, 0.717, 0.08]","[0.048, -0.391, -0.241, -0.122]","[0.064, -0.02, 1.154, 1.954]","[0.04, 0.252, 0.721, 0.024]","[0.091, -0.159, -1.172, -1.901]","[0.019, -1.009, -0.371, 0.313]","[0.03, 0.556, -0.173, -0.263]","[0.098, 2.149, 2.227, 1.276]","[0.102, -0.944, -2.037, -0.789]"
3,"[0.019, 0.823, -1.008, -1.003]","[0.037, -0.253, 0.247, 1.065]","[0.035, 0.417, 0.819, 0.125]","[0.036, -0.641, 0.653, -0.375]","[0.04, 0.393, 0.022, 0.281]","[0.038, -1.31, -1.137, 0.158]","[0.033, -0.524, -0.266, -0.287]","[0.061, 0.203, 0.245, 0.751]","[0.034, -0.349, 1.205, 0.16]","[0.012, 1.587, -0.432, -0.529]"
4,"[0.105, 1.302, 1.475, 1.181]","[0.051, 0.799, -0.083, 0.452]","[0.046, 0.419, 1.304, 0.021]","[0.093, -0.622, -1.434, -0.959]","[0.029, 0.904, 0.079, 0.404]","[0.071, 0.016, 1.815, 1.381]","[0.117, -1.253, -1.321, -1.404]","[0.053, -0.473, -1.11, -0.019]","[0.033, 0.633, 1.469, 0.806]","[0.121, -1.006, -1.474, -1.143]"
5,"[0.054, 1.937, 0.364, 1.705]","[0.034, 0.164, -0.239, -0.201]","[0.045, -0.296, 0.188, -0.791]","[0.078, 0.765, 1.03, 1.459]","[0.046, -0.522, -0.949, -0.259]","[0.048, -0.368, 0.34, 0.266]","[0.054, -0.286, -0.6, 0.206]","[0.063, 0.755, 1.045, 0.854]","[0.154, -1.619, -1.083, -1.807]","[0.058, 0.103, 0.537, -0.798]"
6,"[0.062, 0.817, 0.845, -0.189]","[0.062, 0.264, -0.278, 0.31]","[0.142, 0.706, -0.29, -1.241]","[0.082, 0.653, 0.445, 0.23]","[0.086, -0.617, -0.573, 0.103]","[0.089, -0.997, -0.022, -0.141]","[0.051, 0.467, -0.162, 1.239]","[0.077, 0.26, -0.518, 0.105]","[0.055, -0.252, 0.065, 0.223]","[0.02, -0.575, 1.214, 0.086]"
7,"[0.087, 0.279, 0.943, 1.04]","[0.146, -1.273, -1.682, 0.048]","[0.044, 0.721, 2.434, 0.438]","[0.055, 1.465, -0.563, -0.468]","[0.042, 0.151, 0.459, 0.328]","[0.062, 0.418, -0.752, -0.188]","[0.148, -1.542, -0.554, -2.104]","[0.05, -0.108, 0.389, 0.868]","[0.045, -0.569, -0.251, 0.067]","[0.075, 1.213, 0.33, 0.726]"
8,"[0.055, -0.496, -0.397, 0.424]","[0.126, -1.716, -1.224, -1.667]","[0.084, -0.658, -0.856, 0.062]","[0.057, 0.178, 0.17, 0.59]","[0.08, 0.489, 0.153, 0.711]","[0.082, 1.762, 1.164, 1.532]","[0.07, 0.842, 0.245, -0.124]","[0.121, 0.325, 0.491, 0.928]","[0.042, -0.524, 0.99, -0.457]","[0.08, 0.595, 0.061, -1.204]"
9,"[0.037, 0.978, 0.462, -0.692]","[0.055, 0.155, -1.261, -1.65]","[0.05, -0.033, 2.176, -0.233]","[0.051, 0.241, -0.504, 0.014]","[0.055, -0.836, -0.354, -1.145]","[0.085, -0.644, -1.008, -0.563]","[0.039, -0.09, 0.509, -0.033]","[0.007, 0.002, 0.159, 0.454]","[-0.013, -0.127, -0.492, 2.09]","[0.021, 0.742, 0.703, 2.146]"
